In [1]:
# import apache_beam as beam
# from random import randint


# class SplitRow(beam.DoFn):
#     def process(self, element):
#         return [element.split(",")]


# p = beam.Pipeline()

# (
#     p
#     | beam.io.ReadFromText("../../data/data.csv", skip_header_lines=1)
#     | "Split CSV" >> beam.ParDo(SplitRow())
#     | "Add visits" >> beam.Map(lambda r: ",".join([r[0], str(randint(0, 100))]))
#     | beam.io.WriteToText(file_path_prefix="../../data/data_side_input", file_name_suffix=".csv", header="Name,Visits")
# )

# Run to regenerate the side input
# p.run()


In [4]:
import apache_beam as beam
import csv

p = beam.Pipeline()

side_input = dict()

with open("../../data/data_side_input.csv") as f:
    reader = csv.DictReader(f)
    next(reader)
    for row in reader:
        side_input[row['Name']] = row['Visits']


class FindUsersWithNoVisits(beam.DoFn):
    def process(self, element, side_input):
        row = element.split(",")
        visits = side_input.get(row[0])
        if visits != None and int(visits) < 1:
            return [element]


(
    p
    | beam.io.ReadFromText("../../data/data.csv", skip_header_lines=1)
    | "Find users with 0 visits" >> beam.ParDo(FindUsersWithNoVisits(), side_input)
    | beam.Map(print)
)

p.run()


Andrea Jackson,59,4916 Michelle Rest,New Gracefort,Minnesota,64557,126.020734,-34.8991835
Richard Martin,48,22754 Linda Lodge,Port Sharon,Wisconsin,49224,174.995651,-87.054590
Morgan Heath,44,035 Edward Views Apt. 016,Andersonview,New Hampshire,27516,-26.058075,58.475900
Ryan Fletcher,30,072 Nathaniel Rapids,North Shannon,South Dakota,95096,-68.778417,29.5621155
Karen Mitchell,35,52500 Santiago Land,West Victoria,South Carolina,82346,-145.259749,-4.6940235
Kenneth Reyes,22,143 Daniel Camp Suite 466,East Leonardshire,Texas,51956,58.409274,-63.3363975
Michelle Andrews,58,6503 Melissa Extensions Suite 368,South Jason,North Dakota,55065,-148.418236,68.129798
Roberto Kelly,51,7904 Nicole Motorway Apt. 903,Lake Kristaport,Hawaii,96507,112.552682,-49.642809
Shannon Haney,31,3262 Torres Points Apt. 658,South Robert,Montana,64408,42.755517,32.129678
Derrick Hale,37,252 Kathryn Squares Suite 980,Pamelachester,New Mexico,81859,167.491012,-78.670019
Linda Nguyen,56,6422 Bryant Unions,Richardsonche